In [1]:
# 
import bikes_data

bikes = bikes_data.data_preprocessing() 
bikes.head(3)

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리,요일,일자,대여시간대,주말구분,대여구,대여점위도,대여점경도
0,SPB-22040,2019-06-03 08:49:27,646,장한평역 1번출구 (국민은행앞),1,2019-06-03 09:17:10,3,중랑센터,14,27,1330.0,월,3,8,평일,동대문구,37.561966,127.064506
1,SPB-07446,2019-06-03 08:33:22,526,용답토속공원 앞,8,2019-06-03 09:27:16,3,중랑센터,14,54,1180.0,월,3,8,평일,성동구,37.563511,127.056725
2,SPB-20387,2019-06-05 08:27:47,646,장한평역 1번출구 (국민은행앞),1,2019-06-05 08:41:31,3,중랑센터,2,12,1930.0,수,5,8,평일,동대문구,37.561966,127.064506


# 인기있는 대여소 분석
- 주말에 인기있는 대여소 top 50
- 평일에 인기있는 대여소 top 50

In [3]:
# 인기있는 대여소 분석
# 주말에 인기있는 대여소 top 50
# 평일에 인기있는 대여소 top 50

bikes.columns

Index(['자전거번호', '대여일시', '대여 대여소번호', '대여 대여소명', '대여거치대', '반납일시', '반납대여소번호',
       '반납대여소명', '반납거치대', '이용시간', '이용거리', '요일', '일자', '대여시간대', '주말구분', '대여구',
       '대여점위도', '대여점경도'],
      dtype='object')

In [6]:
rent_bike = bikes.pivot_table(index=['대여 대여소명','대여점위도','대여점경도'],
                              columns=['주말구분'],
                              values='자전거번호',
                              aggfunc='count')
rent_bike.head()

,,주말구분,주말,평일
대여 대여소명,대여점위도,대여점경도,,
(구)신한은행 중랑교지점,37.598591,127.079819,391,832
3호선 매봉역 3번출구앞,37.486767,127.046761,443,1049
419민주묘지역,37.649673,127.013451,547,946
8삼호@ 2동 ( 간선도로),37.493759,127.045898,111,316
9호선종합운동장역 9번출구,37.511280,127.078239,668,1136


In [8]:
rent_bike.info()
#  1456 entries 중에 상위 50개만 하고 싶다.

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1456 entries, ('(구)신한은행 중랑교지점', np.float64(37.59859085), np.float64(127.0798187)) to ('희성오피앙', np.float64(37.68031311), np.float64(127.055069))
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   주말      1456 non-null   int64
 1   평일      1456 non-null   int64
dtypes: int64(2)
memory usage: 65.3+ KB


In [12]:
rent_bike.sort_values('주말', ascending=False).head(3)

,,주말구분,주말,평일
대여 대여소명,대여점위도,대여점경도,,
뚝섬유원지역 1번출구 앞,37.531860,127.067192,5777,6840
여의나루역 1번출구 앞,37.527157,126.931900,5286,6241
"고속터미널역 8-1번, 8-2번 출구 사이",37.506199,127.003944,3932,4556


In [29]:
# 인덱스를 풀자

weekend_house50 = rent_bike.nlargest(50,'주말')[['주말']].reset_index()

 #‘주말’ 열을 기준으로 상위 50개 행을 선택, [['주말']] 열만 추출, 
 # 새로운 DataFrame의 인덱스를 재설정. 이렇게 하면 ‘index’ 열이 추가

weekend_house50.head()

주말구분,대여 대여소명,대여점위도,대여점경도,주말
0,뚝섬유원지역 1번출구 앞,37.531860,127.067192,5777
1,여의나루역 1번출구 앞,37.527157,126.931900,5286
2,"고속터미널역 8-1번, 8-2번 출구 사이",37.506199,127.003944,3932
3,봉림교 교통섬,37.484085,126.926888,3699
4,롯데월드타워(잠실역2번출구 쪽),37.513126,127.100960,3231


In [24]:
# 주말
import folium

# 지도의 중심위치를 정한다.
lat = bikes['대여점위도'].mean()
lon = bikes['대여점경도'].mean()
center = [lat, lon]
map1 = folium.Map(location=center, zoom_start=11)


for i in weekend_house50.index :
    sub_lat = weekend_house50.loc[i,'대여점위도']
    sub_lon = weekend_house50.loc[i,'대여점경도']
    name = weekend_house50.loc[i,'대여 대여소명']
    folium.Marker(location=[sub_lat,sub_lon],
                  popup=name).add_to(map1)

map1


주말구분,대여 대여소명,대여점위도,대여점경도,평일
0,뚝섬유원지역 1번출구 앞,37.531860,127.067192,6840
1,여의나루역 1번출구 앞,37.527157,126.931900,6241
2,봉림교 교통섬,37.484085,126.926888,5986
3,홍대입구역 2번출구 앞,37.557438,126.923821,5617
4,롯데월드타워(잠실역2번출구 쪽),37.513126,127.100960,5383


In [34]:
# 평일 인기 있는 대여소 시각화 iterrows()

weekday_house50 = rent_bike.nlargest(50,'평일')[['평일']].reset_index()
# weekday_house50.head()

# 지도의 중심위치를 정한다.
lat = bikes['대여점위도'].mean()
lon = bikes['대여점경도'].mean()
center = [lat, lon]
# map2 = folium.Map(location=center, zoom_start=11)


for i , row in weekday_house50.iterrows():
    sub_lat = row['대여점위도']
    sub_lon = row['대여점경도']
    name = row['대여 대여소명']

    folium.Marker(location=[sub_lat, sub_lon],
                  popup=name,
                  icon=folium.Icon(color='r',icon='bicycle', prefix='fa')                  
                  ).add_to(map1)
    
map1

C:\Users\USER\AppData\Local\Temp\ipykernel_1868\2858833980.py:20: UserWarning: color argument of Icon should be one of: {'pink', 'lightblue', 'black', 'darkblue', 'white', 'beige', 'gray', 'darkpurple', 'green', 'lightgreen', 'purple', 'red', 'cadetblue', 'lightgray', 'blue', 'orange', 'darkred', 'darkgreen', 'lightred'}.
  icon=folium.Icon(color='r',icon='bicycle', prefix='fa')
